## Data Pipline 

Preparing individual dataset for merging  

Train and transactions obtained from eda.ipynb 
Holidays, Oil , Items , and stores obtained from  ... 

In [5]:
import pandas as pd
import seaborn as sns 
import numpy as np 
import datetime
import os 
from matplotlib import pyplot as plt
import random
import dask.dataframe as dd
from sklearn import preprocessing

Train

In [6]:
# optimizing size of data types 
dtype_dict={"id":'int32',
            "store_nbr":'int8',
            "item_nbr":'int32',
            "unit_sales":'float32',
            
           }
df1 = pd.read_csv("/Volumes/HD David/COSC490/favorita-grocery-sales-forecasting/train.csv",dtype=dtype_dict,parse_dates=['date'])

/Users/davidpinosproano/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df1['onpromotion'] = df1['onpromotion'].fillna(0)
df1['onpromotion'] = df1['onpromotion'].astype('int8')
df1.dtypes

id                      int32
date           datetime64[ns]
store_nbr                int8
item_nbr                int32
unit_sales            float32
onpromotion              int8
dtype: object

Transactions 

In [8]:
dtype_dict={"id":'int32',
            "store_nbr":'int8',
            "transactions":'int32',

           }
transactions= pd.read_csv("/Volumes/HD David/COSC490/favorita-grocery-sales-forecasting/transactions.csv",dtype=dtype_dict,parse_dates=['date'])


In [9]:
transactions.dtypes


date            datetime64[ns]
store_nbr                 int8
transactions             int32
dtype: object

Holidays

In [10]:

holidays = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/holidays_clean.csv", parse_dates = ['date'])
holidays = holidays.drop(columns = ['Unnamed: 0'])

In [76]:
# Cleaning holidays table to generate a df with columns date and holiday(bool)

holidays = holidays.drop(holidays[holidays.transferred== True ].index)
holidays = holidays[(holidays.type == 'Holiday') | (holidays.type == 'Bridge')]
holidays = holidays[holidays.locale == 'National']
holidays['holiday'] = 1


AttributeError: 'DataFrame' object has no attribute 'transferred'

In [74]:
#list of national holidays 
holidays = holidays[['date','holiday']]

Oil 

In [11]:
oil = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/oil_clean.csv", parse_dates = ['date'])
oil = oil.drop(columns = ['Unnamed: 0'])

Items 

In [12]:
items = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/items_clean.csv")
items = items.drop(columns = ['Unnamed: 0'])

Stores

In [13]:
stores = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/stores_clean.csv")
stores = stores.drop(columns = ['Unnamed: 0'])

# Sample from the Data 

In [14]:
# sampling 10% of the training dataset
random.seed(100)
sample1 = df1.sample(n= round(len(df1)*0.1))
sample1 = sample1.reset_index(drop =True)

# Join Tables

In [15]:
# worked well on merging trasnactions with 10 percent of the data 
train_tran = sample1.merge(transactions,on = ['date','store_nbr'],how = 'inner')

In [16]:
train_tran_stores = train_tran.merge(stores,on = "store_nbr",how = "inner")

In [17]:
train_tran_stores_oil = train_tran_stores.merge(oil, on = "date",how = "inner")

In [18]:
train_tran_stores_oil_items = train_tran_stores_oil.merge(items,on = "item_nbr", how = "inner")

In [77]:
#holiday yes no has to be binary 

# holidays[['date','locale']].sort_values(by = 'date')
train_tran_stores_oil_items_hol = train_tran_stores_oil_items.merge(holidays, on = 'date',how = 'left')


# Exploration
-----------------------

In [79]:
print("sample")
print(len(sample1))
print("transactions")
print(len(train_tran))
print("sotores")
print( len(train_tran_stores))
print("oil")
print( len(train_tran_stores_oil))
print("items")
print(len(train_tran_stores_oil_items))
print("hol")
print( len(train_tran_stores_oil_items_hol))


sample
12549704
transactions
12528227
sotores
12528227
oil
12528227
items
12528227
hol
12528227


In [21]:
# doing a left join produces 1 percent more data due to more than one holiday landing on the same day. 
12528238/12549704

0.9982895214102261

Final feature engineering 

In [89]:
train_tran_stores_oil_items_hol['holiday'] = train_tran_stores_oil_items_hol['holiday'].fillna(0)

In [96]:
# this is the output after removing cluster,type,city,state,and class using a sample of 10 percent of the data. 

final_sample = train_tran_stores_oil_items_hol.drop(columns = ['cluster','type','city','state','class'])


In [97]:
# make categorical variables into numerical
# using sklearn encoding
# we need to categorize  store number , item _nbr 
final_sample['item_nbr'].value_counts()

314384     8407
364606     8268
261052     8238
265559     8237
559870     8233
           ... 
1976284       1
2121690       1
2011459       1
2123711       1
2011451       1
Name: item_nbr, Length: 4028, dtype: int64

In [99]:
final_sample

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,oil_price,family,perishable,holiday
0,56187408,2015-09-10,5,1324591,2.0,0,1259,45.850,GROCERY I,0,0.0
1,56193188,2015-09-10,8,1324591,4.0,0,2520,45.850,GROCERY I,0,0.0
2,56227011,2015-09-10,34,1324591,3.0,0,1982,45.850,GROCERY I,0,0.0
3,56237409,2015-09-10,41,1324591,4.0,0,957,45.850,GROCERY I,0,0.0
4,56255141,2015-09-10,50,1324591,1.0,0,1976,45.850,GROCERY I,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
12528222,125269345,2017-08-13,45,2011468,2.0,0,4054,45.765,GROCERY I,0,0.0
12528223,125230200,2017-08-13,24,2123791,2.0,0,1520,45.765,GROCERY I,0,0.0
12528224,125383877,2017-08-14,49,2011448,2.0,0,2916,47.590,GROCERY I,0,0.0
12528225,125384120,2017-08-14,49,2123790,1.0,0,2916,47.590,"LIQUOR,WINE,BEER",0,0.0


In [100]:
# scikit learn exploration 
from sklearn.preprocessing import LabelEncoder

In [103]:
final_sample['family_cat'] = LabelEncoder().fit_transform(final_sample['family'])

In [104]:
final_sample

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,oil_price,family,perishable,holiday,family_cat
0,56187408,2015-09-10,5,1324591,2.0,0,1259,45.850,GROCERY I,0,0.0,12
1,56193188,2015-09-10,8,1324591,4.0,0,2520,45.850,GROCERY I,0,0.0,12
2,56227011,2015-09-10,34,1324591,3.0,0,1982,45.850,GROCERY I,0,0.0,12
3,56237409,2015-09-10,41,1324591,4.0,0,957,45.850,GROCERY I,0,0.0,12
4,56255141,2015-09-10,50,1324591,1.0,0,1976,45.850,GROCERY I,0,0.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...
12528222,125269345,2017-08-13,45,2011468,2.0,0,4054,45.765,GROCERY I,0,0.0,12
12528223,125230200,2017-08-13,24,2123791,2.0,0,1520,45.765,GROCERY I,0,0.0,12
12528224,125383877,2017-08-14,49,2011448,2.0,0,2916,47.590,GROCERY I,0,0.0,12
12528225,125384120,2017-08-14,49,2123790,1.0,0,2916,47.590,"LIQUOR,WINE,BEER",0,0.0,22
